# XxxWorkflow.wdl test results to Xcelsheet ".tsv", ".tsv" or ".xlsx"
    * The python working model of a spreadsheet is the pandas DataFrame.
    * This code parses the results directory files content into a DataFrame.
    * The options are then to write the DataFrame to a file or display on the command line or both.

In [3]:
import os
import sys
import pandas as pd
import numpy as np

#       Set and verify the Results Directory for this notebook
x_dir = '/Users/yo/zzIForge/fullyJan10/'
if not os.path.isdir(x_dir):
    print('directory not found\n', x_dir)

In [57]:
def get_test_results_dataframe(x_dir):
    """ Usage: return_codes_dataframe = get_test_results_dataframe(x_dir)
    args:
        x_dir:         the directory with the "call_..." subdirectories (else you get nothing)
        
    returns:
        rc_df:         pandas dataframe with the return codes and size of various output files
    """
    DATAFRAME_DEFAULT_EMPTY_VALUE = 'unk'
    FAILED_RETURN_CODE_READ = '-1'
    good_return_codes_list = ['0', '0\n']
    check_files_dict = {'stderr':['ERROR', 'error', 'Error'], 'stdout':['START', 'Finished']}
    
    #     This variable might be compared to the "call" entries in the .wdl tree.
    call_dirs = os.listdir(x_dir)
    call_dir_list = []
    call_dir_count = 0
    #     Get the rows list - parse the directories that begin with "call" vs getting them from the .wdl files
    for call_dir in call_dirs:
        if os.path.isdir(os.path.join(call_dir, x_dir)) and call_dir[0:4] == 'call':
            call_dir_count += 1
            call_dir_list.append(call_dir)
    
    #     Create the list of things that will be reported in each call directory and initialize the dataframe
    cols_list = ['rc', 'bam', 'bam.bai', 'stderr', 'stdout']
    rc_df = pd.DataFrame(index=call_dir_list,columns=cols_list).fillna(DATAFRAME_DEFAULT_EMPTY_VALUE)
    
    #     Check the directories in this tree against the column list
    for dir_name, dir_list, files_list in os.walk(x_dir):
        for filename in files_list:
            full_filename = os.path.join(dir_name, filename)
            if filename in cols_list:
                if filename == 'rc':
                    with open(full_filename, 'r') as fh:
                        lines = fh.readlines()
                        
                    if len(lines) > 0:
                        for call_dir in call_dir_list:
                            if call_dir in dir_name:
                                if lines[0] in good_return_codes_list:
                                    rc_df.loc[call_dir, 'rc'] = str(lines[0]).strip()
                                else:
                                    try:
                                        rc_df.loc[call_dir, 'rc'] = str(lines[0]).strip()
                                    except:
                                        rc_df.loc[call_dir, 'rc'] = FAILED_RETURN_CODE_READ
                                        pass
                                    
            if filename in list(check_files_dict.keys()):
                for call_dir in call_dir_list:
                    if call_dir in dir_name:
                        with open(full_filename, 'r') as fh:
                            lines = fh.readlines()

                        if len(lines) > 0:
                            for line in lines:
                                for check_word in check_files_dict[filename]:
                                    if check_word in line:
                                        rc_df.loc[call_dir, filename] = check_word
                                        continue
                
            # Report the File sizes - code version pending full list of required files
            else:
                for call_dir in call_dir_list:
                    if call_dir in dir_name:
                        fname, fext = os.path.splitext(filename)
                        this_file_data = os.stat(full_filename)
                        if fext[1:] == 'bam':
                            rc_df.loc[call_dir, 'bam'] = str(this_file_data.st_size)
                        elif fext[1:] == 'bai':
                            rc_df.loc[call_dir, 'bam.bai'] = str(this_file_data.st_size)
                                                            
    return rc_df                 

In [58]:
somedf = get_test_results_dataframe(x_dir)
somedf

,rc,bam,bam.bai,stderr,stdout
call-DHVC,0,unk,unk,unk,unk
call-realign,0,65349394,1431360,unk,START
call-align,0,59500466,1431328,unk,Finished
call-bqsr,0,67462520,1431360,unk,Finished
call-dedup,0,65322724,1431360,unk,Finished
call-haplotype,0,67462520,1431360,unk,Finished
call-merge,0,59500466,1431328,unk,unk
call-vqsr,0,unk,unk,unk,Finished
call-trimseq,0,unk,unk,unk,Finished
call-DAB,0,65349394,1431360,unk,unk


In [4]:
#                                  Be convinced of similar names
json_dir = '/Users/yo/zzIForge/Jsons'
json_farameters_file = 'GermlineMasterWorkflow.FilledIn.json'
import knpackage.toolbox as kn
json_parameters = kn.get_run_parameters(json_dir, json_farameters_file)
dakine = 'WorkflowJson'
lineno = 0
for k, v in json_parameters.items():
    if dakine in k:
        print('%i %30s\n\t%s\n'%(lineno, k, v))
    lineno += 1

7 GermlineMasterWF.DAB.WorkflowJson
	/projects/bioinformatics/DEL/Jsons/GermlineMasterWorkflow.FilledIn.json

14 GermlineMasterWF.DHVC.WorkflowJson
	/projects/bioinformatics/DEL/Jsons/GermlineMasterWorkflow.FilledIn.json

